In [26]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = torch.load("../normalizeddataset.pt")


lengths = [
    int(len(dataset) * 0.8),
    int(len(dataset) * 0.1),
    int(len(dataset) * 0.1) + 1
]

# trainset, valset, testset = random_split(dataset, lengths)
# image_datasets = {'train': trainset, 'val': valset, 'test': testset}
# dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
#               for x in ['train', 'val', 'test']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}  


NC = []
AD = []
for data in dataset:
    if data[1] == 0:
        NC.append(data)
    else:
        AD.append(data)
        
NC = [sample[0] for sample in NC]
NCgan1 = [torch.unsqueeze(sample[0][0], 0) for sample in NC]
NCgan2 = [torch.unsqueeze(sample[1][0], 0) for sample in NC]
NCgan3 = [torch.unsqueeze(sample[2][0], 0) for sample in NC]

batch_size = 16
dataloader1 = DataLoader(NCgan1[:-1], batch_size=batch_size, shuffle=True, num_workers=4)
dataloader2 = DataLoader(NCgan2[:-1], batch_size=batch_size, shuffle=True, num_workers=4)
dataloader3 = DataLoader(NCgan3[:-1], batch_size=batch_size, shuffle=True, num_workers=4)

In [27]:
import os
import numpy as np
import math
import sys
from tqdm.notebook import tqdm
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

n_epochs = 200
lr=0.00005
n_cpu = 8
latent_dim = 100
img_size = 64
channels = 1
n_critic = 2
clip_value = 0.01
sample_interval = 400


img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity


# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=lr)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [41]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()

def train_gan(dataloader):
    batches_done = 0
    for epoch in range(n_epochs):

        for i, imgs in enumerate(dataloader):

            real_imgs = Variable(imgs.type(Tensor))

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Sample noise as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

            # Generate a batch of images
            fake_imgs = generator(z).detach()
            # Adversarial loss
            loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

            loss_D.backward()
            optimizer_D.step()

            # Clip weights of discriminator
            for p in discriminator.parameters():
                p.data.clamp_(-clip_value, clip_value)

            # Train the generator every n_critic iterations
            if i % n_critic == 0:

                # -----------------
                #  Train Generator
                # -----------------

                optimizer_G.zero_grad()

                # Generate a batch of images
                gen_imgs = generator(z)
                # Adversarial loss
                loss_G = -torch.mean(discriminator(gen_imgs))

                loss_G.backward()
                optimizer_G.step()

                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                    % (epoch, n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
                )

            if batches_done % sample_interval == 0:
                save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
            batches_done += 1
            
train_gan(dataloader3)

[Epoch 0/200] [Batch 0/44] [D loss: -0.048885] [G loss: -0.010611]
[Epoch 0/200] [Batch 2/44] [D loss: 0.002292] [G loss: -0.010627]
[Epoch 0/200] [Batch 4/44] [D loss: 0.005499] [G loss: -0.010612]
[Epoch 0/200] [Batch 6/44] [D loss: 0.006127] [G loss: -0.010639]
[Epoch 0/200] [Batch 8/44] [D loss: 0.006159] [G loss: -0.010632]
[Epoch 0/200] [Batch 10/44] [D loss: 0.005807] [G loss: -0.010619]
[Epoch 0/200] [Batch 12/44] [D loss: 0.004198] [G loss: -0.010611]
[Epoch 0/200] [Batch 14/44] [D loss: 0.006733] [G loss: -0.010595]
[Epoch 0/200] [Batch 16/44] [D loss: 0.006999] [G loss: -0.010603]
[Epoch 0/200] [Batch 18/44] [D loss: 0.005503] [G loss: -0.010629]
[Epoch 0/200] [Batch 20/44] [D loss: 0.005787] [G loss: -0.010607]
[Epoch 0/200] [Batch 22/44] [D loss: 0.007644] [G loss: -0.010623]
[Epoch 0/200] [Batch 24/44] [D loss: 0.005483] [G loss: -0.010629]
[Epoch 0/200] [Batch 26/44] [D loss: 0.006771] [G loss: -0.010637]
[Epoch 0/200] [Batch 28/44] [D loss: 0.007565] [G loss: -0.010610]

[Epoch 6/200] [Batch 0/44] [D loss: 0.005604] [G loss: -0.010607]
[Epoch 6/200] [Batch 2/44] [D loss: 0.006089] [G loss: -0.010610]
[Epoch 6/200] [Batch 4/44] [D loss: 0.005433] [G loss: -0.010611]
[Epoch 6/200] [Batch 6/44] [D loss: 0.007245] [G loss: -0.010622]
[Epoch 6/200] [Batch 8/44] [D loss: 0.005840] [G loss: -0.010610]
[Epoch 6/200] [Batch 10/44] [D loss: 0.005014] [G loss: -0.010588]
[Epoch 6/200] [Batch 12/44] [D loss: 0.007845] [G loss: -0.010618]
[Epoch 6/200] [Batch 14/44] [D loss: 0.004329] [G loss: -0.010608]
[Epoch 6/200] [Batch 16/44] [D loss: 0.005105] [G loss: -0.010629]
[Epoch 6/200] [Batch 18/44] [D loss: 0.001555] [G loss: -0.010642]
[Epoch 6/200] [Batch 20/44] [D loss: 0.005170] [G loss: -0.010615]
[Epoch 6/200] [Batch 22/44] [D loss: 0.007566] [G loss: -0.010602]
[Epoch 6/200] [Batch 24/44] [D loss: 0.005506] [G loss: -0.010623]
[Epoch 6/200] [Batch 26/44] [D loss: 0.006543] [G loss: -0.010632]
[Epoch 6/200] [Batch 28/44] [D loss: 0.005860] [G loss: -0.010600]


[Epoch 12/200] [Batch 0/44] [D loss: 0.005043] [G loss: -0.010614]
[Epoch 12/200] [Batch 2/44] [D loss: 0.007062] [G loss: -0.010619]
[Epoch 12/200] [Batch 4/44] [D loss: 0.006681] [G loss: -0.010643]
[Epoch 12/200] [Batch 6/44] [D loss: 0.003919] [G loss: -0.010614]
[Epoch 12/200] [Batch 8/44] [D loss: 0.006501] [G loss: -0.010617]
[Epoch 12/200] [Batch 10/44] [D loss: 0.006080] [G loss: -0.010604]
[Epoch 12/200] [Batch 12/44] [D loss: 0.005281] [G loss: -0.010624]
[Epoch 12/200] [Batch 14/44] [D loss: 0.007167] [G loss: -0.010627]
[Epoch 12/200] [Batch 16/44] [D loss: 0.007563] [G loss: -0.010637]
[Epoch 12/200] [Batch 18/44] [D loss: 0.004197] [G loss: -0.010631]
[Epoch 12/200] [Batch 20/44] [D loss: 0.006491] [G loss: -0.010626]
[Epoch 12/200] [Batch 22/44] [D loss: 0.006018] [G loss: -0.010607]
[Epoch 12/200] [Batch 24/44] [D loss: 0.006302] [G loss: -0.010621]
[Epoch 12/200] [Batch 26/44] [D loss: 0.004722] [G loss: -0.010588]
[Epoch 12/200] [Batch 28/44] [D loss: 0.007040] [G lo

[Epoch 18/200] [Batch 0/44] [D loss: 0.003780] [G loss: -0.010611]
[Epoch 18/200] [Batch 2/44] [D loss: 0.004973] [G loss: -0.010607]
[Epoch 18/200] [Batch 4/44] [D loss: 0.004803] [G loss: -0.010612]
[Epoch 18/200] [Batch 6/44] [D loss: 0.006407] [G loss: -0.010627]
[Epoch 18/200] [Batch 8/44] [D loss: 0.005681] [G loss: -0.010616]
[Epoch 18/200] [Batch 10/44] [D loss: 0.006252] [G loss: -0.010626]
[Epoch 18/200] [Batch 12/44] [D loss: 0.006251] [G loss: -0.010629]
[Epoch 18/200] [Batch 14/44] [D loss: 0.005141] [G loss: -0.010602]
[Epoch 18/200] [Batch 16/44] [D loss: 0.008099] [G loss: -0.010606]
[Epoch 18/200] [Batch 18/44] [D loss: 0.005457] [G loss: -0.010609]
[Epoch 18/200] [Batch 20/44] [D loss: 0.007361] [G loss: -0.010609]
[Epoch 18/200] [Batch 22/44] [D loss: 0.005565] [G loss: -0.010641]
[Epoch 18/200] [Batch 24/44] [D loss: 0.005687] [G loss: -0.010619]
[Epoch 18/200] [Batch 26/44] [D loss: 0.008268] [G loss: -0.010625]
[Epoch 18/200] [Batch 28/44] [D loss: 0.004847] [G lo

[Epoch 24/200] [Batch 0/44] [D loss: 0.004222] [G loss: -0.010628]
[Epoch 24/200] [Batch 2/44] [D loss: 0.005875] [G loss: -0.010607]
[Epoch 24/200] [Batch 4/44] [D loss: 0.004574] [G loss: -0.010613]
[Epoch 24/200] [Batch 6/44] [D loss: 0.004548] [G loss: -0.010605]
[Epoch 24/200] [Batch 8/44] [D loss: 0.007300] [G loss: -0.010618]
[Epoch 24/200] [Batch 10/44] [D loss: 0.005761] [G loss: -0.010609]
[Epoch 24/200] [Batch 12/44] [D loss: 0.005164] [G loss: -0.010617]
[Epoch 24/200] [Batch 14/44] [D loss: 0.004592] [G loss: -0.010597]
[Epoch 24/200] [Batch 16/44] [D loss: 0.006040] [G loss: -0.010626]
[Epoch 24/200] [Batch 18/44] [D loss: 0.006689] [G loss: -0.010619]
[Epoch 24/200] [Batch 20/44] [D loss: 0.006769] [G loss: -0.010635]
[Epoch 24/200] [Batch 22/44] [D loss: 0.004482] [G loss: -0.010607]
[Epoch 24/200] [Batch 24/44] [D loss: 0.005315] [G loss: -0.010622]
[Epoch 24/200] [Batch 26/44] [D loss: 0.005651] [G loss: -0.010586]
[Epoch 24/200] [Batch 28/44] [D loss: 0.003865] [G lo

...

[Epoch 174/200] [Batch 0/44] [D loss: 0.005906] [G loss: -0.010616]
[Epoch 174/200] [Batch 2/44] [D loss: 0.005982] [G loss: -0.010611]
[Epoch 174/200] [Batch 4/44] [D loss: 0.006296] [G loss: -0.010605]
[Epoch 174/200] [Batch 6/44] [D loss: 0.005978] [G loss: -0.010596]
[Epoch 174/200] [Batch 8/44] [D loss: 0.005763] [G loss: -0.010610]
[Epoch 174/200] [Batch 10/44] [D loss: 0.005526] [G loss: -0.010633]
[Epoch 174/200] [Batch 12/44] [D loss: 0.005799] [G loss: -0.010627]
[Epoch 174/200] [Batch 14/44] [D loss: 0.005675] [G loss: -0.010616]
[Epoch 174/200] [Batch 16/44] [D loss: 0.005548] [G loss: -0.010607]
[Epoch 174/200] [Batch 18/44] [D loss: 0.006840] [G loss: -0.010597]
[Epoch 174/200] [Batch 20/44] [D loss: 0.004627] [G loss: -0.010631]
[Epoch 174/200] [Batch 22/44] [D loss: 0.003928] [G loss: -0.010608]
[Epoch 174/200] [Batch 24/44] [D loss: 0.004060] [G loss: -0.010629]
[Epoch 174/200] [Batch 26/44] [D loss: 0.006300] [G loss: -0.010631]
[Epoch 174/200] [Batch 28/44] [D loss: 

[Epoch 180/200] [Batch 0/44] [D loss: 0.004848] [G loss: -0.010616]
[Epoch 180/200] [Batch 2/44] [D loss: 0.004516] [G loss: -0.010624]
[Epoch 180/200] [Batch 4/44] [D loss: 0.006966] [G loss: -0.010607]
[Epoch 180/200] [Batch 6/44] [D loss: 0.005975] [G loss: -0.010603]
[Epoch 180/200] [Batch 8/44] [D loss: 0.005291] [G loss: -0.010600]
[Epoch 180/200] [Batch 10/44] [D loss: 0.006528] [G loss: -0.010605]
[Epoch 180/200] [Batch 12/44] [D loss: 0.004911] [G loss: -0.010631]
[Epoch 180/200] [Batch 14/44] [D loss: 0.007831] [G loss: -0.010627]
[Epoch 180/200] [Batch 16/44] [D loss: 0.004834] [G loss: -0.010626]
[Epoch 180/200] [Batch 18/44] [D loss: 0.005195] [G loss: -0.010614]
[Epoch 180/200] [Batch 20/44] [D loss: 0.005993] [G loss: -0.010639]
[Epoch 180/200] [Batch 22/44] [D loss: 0.005247] [G loss: -0.010618]
[Epoch 180/200] [Batch 24/44] [D loss: 0.004291] [G loss: -0.010625]
[Epoch 180/200] [Batch 26/44] [D loss: 0.003858] [G loss: -0.010636]
[Epoch 180/200] [Batch 28/44] [D loss: 

[Epoch 186/200] [Batch 0/44] [D loss: 0.003978] [G loss: -0.010622]
[Epoch 186/200] [Batch 2/44] [D loss: 0.008369] [G loss: -0.010641]
[Epoch 186/200] [Batch 4/44] [D loss: 0.006897] [G loss: -0.010620]
[Epoch 186/200] [Batch 6/44] [D loss: 0.006356] [G loss: -0.010610]
[Epoch 186/200] [Batch 8/44] [D loss: 0.005822] [G loss: -0.010619]
[Epoch 186/200] [Batch 10/44] [D loss: 0.004372] [G loss: -0.010615]
[Epoch 186/200] [Batch 12/44] [D loss: 0.007288] [G loss: -0.010634]
[Epoch 186/200] [Batch 14/44] [D loss: 0.004666] [G loss: -0.010620]
[Epoch 186/200] [Batch 16/44] [D loss: 0.005612] [G loss: -0.010597]
[Epoch 186/200] [Batch 18/44] [D loss: 0.005370] [G loss: -0.010623]
[Epoch 186/200] [Batch 20/44] [D loss: 0.005893] [G loss: -0.010618]
[Epoch 186/200] [Batch 22/44] [D loss: 0.006491] [G loss: -0.010626]
[Epoch 186/200] [Batch 24/44] [D loss: 0.005747] [G loss: -0.010625]
[Epoch 186/200] [Batch 26/44] [D loss: 0.005476] [G loss: -0.010606]
[Epoch 186/200] [Batch 28/44] [D loss: 

[Epoch 192/200] [Batch 0/44] [D loss: 0.004372] [G loss: -0.010600]
[Epoch 192/200] [Batch 2/44] [D loss: 0.004078] [G loss: -0.010618]
[Epoch 192/200] [Batch 4/44] [D loss: 0.006381] [G loss: -0.010621]
[Epoch 192/200] [Batch 6/44] [D loss: 0.003643] [G loss: -0.010620]
[Epoch 192/200] [Batch 8/44] [D loss: 0.007367] [G loss: -0.010618]
[Epoch 192/200] [Batch 10/44] [D loss: 0.004528] [G loss: -0.010640]
[Epoch 192/200] [Batch 12/44] [D loss: 0.006784] [G loss: -0.010628]
[Epoch 192/200] [Batch 14/44] [D loss: 0.007370] [G loss: -0.010607]
[Epoch 192/200] [Batch 16/44] [D loss: 0.006782] [G loss: -0.010606]
[Epoch 192/200] [Batch 18/44] [D loss: 0.007283] [G loss: -0.010598]
[Epoch 192/200] [Batch 20/44] [D loss: 0.004692] [G loss: -0.010614]
[Epoch 192/200] [Batch 22/44] [D loss: 0.003889] [G loss: -0.010629]
[Epoch 192/200] [Batch 24/44] [D loss: 0.005408] [G loss: -0.010622]
[Epoch 192/200] [Batch 26/44] [D loss: 0.007214] [G loss: -0.010619]
[Epoch 192/200] [Batch 28/44] [D loss: 

[Epoch 198/200] [Batch 0/44] [D loss: 0.005831] [G loss: -0.010615]
[Epoch 198/200] [Batch 2/44] [D loss: 0.007165] [G loss: -0.010607]
[Epoch 198/200] [Batch 4/44] [D loss: 0.004847] [G loss: -0.010605]
[Epoch 198/200] [Batch 6/44] [D loss: 0.006707] [G loss: -0.010630]
[Epoch 198/200] [Batch 8/44] [D loss: 0.004408] [G loss: -0.010595]
[Epoch 198/200] [Batch 10/44] [D loss: 0.005707] [G loss: -0.010624]
[Epoch 198/200] [Batch 12/44] [D loss: 0.004157] [G loss: -0.010621]
[Epoch 198/200] [Batch 14/44] [D loss: 0.007536] [G loss: -0.010622]
[Epoch 198/200] [Batch 16/44] [D loss: 0.003958] [G loss: -0.010628]
[Epoch 198/200] [Batch 18/44] [D loss: 0.005873] [G loss: -0.010602]
[Epoch 198/200] [Batch 20/44] [D loss: 0.006679] [G loss: -0.010640]
[Epoch 198/200] [Batch 22/44] [D loss: 0.005788] [G loss: -0.010634]
[Epoch 198/200] [Batch 24/44] [D loss: 0.005695] [G loss: -0.010613]
[Epoch 198/200] [Batch 26/44] [D loss: 0.005146] [G loss: -0.010627]
[Epoch 198/200] [Batch 28/44] [D loss: 